# Téléchargement des packages

In [10]:
!pip install textblob
!pip install sklearn

You should consider upgrading via the 'pip install --upgrade pip' command.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [13]:
# numeric module
import pandas as pd
import numpy as np
import scipy as sp

#machine learning module
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

# text processing module
from textblob import TextBlob, Word
from nltk.stem.porter import PorterStemmer

#import heapq
#from collections import Counter, OrderedDict

# wordcloud visualization
#from wordcloud import WordCloud

# Visualization
#import seaborn as sns
#import matplotlib.pyplot as plt

# Analyse rapide du dataset

In [14]:
df = pd.read_csv("yelp_academic_dataset_review_Auto Repair.csv")
df.head()

,review_id,business_id,stars_x,stars_y,date,city,text,categories
0,JN0tmjiksWfr4i2NIFuPXw,pLZ9oZM8c6MNbRlg06lBPg,1,3.0,2016-08-03,Las Vegas,THEY SELL WRECKED VEHICLES! \nI purchased a 20...,Auto Repair;Car Dealers;Automotive;Auto Parts ...
1,lFmz61DNRzBPnrwH_cF42g,J0KZR0DDnvj73D-xmbh4uA,4,4.0,2017-06-15,Las Vegas,I stopped in here recently as I had to buy a t...,Auto Parts & Supplies;Auto Repair;Automotive
2,R9p3ug8zNPvawYX_jsO0UA,Tknj3u-3CZTI-q52saTAIA,1,1.0,2012-09-12,Toronto,"If you want the ""I don't give a shit "" attitud...",Car Dealers;Auto Repair;Automotive
3,FMXER_6R6URYBpV6TYPn9A,rV1nkTx58v_q7YGDkGzfxQ,5,5.0,2016-06-10,Las Vegas,where to start...\na few months ago I slammed....,Auto Repair;Automotive
4,bJN2ZPa0jF4xgR1gPKZP3A,hCXknMQalY1QQZrdZGuBHg,3,3.5,2017-01-24,Las Vegas,I have mixed feelings here. I recently purchas...,Car Dealers;Auto Repair;Automotive


In [17]:
df = df[(df['stars_x'] == 5) | (df['stars_x'] == 1)]
#only keep rated 1 and rated 5

In [18]:
df[df['stars_x'] == 5].head(3)

,review_id,business_id,stars_x,stars_y,date,city,text,categories
3,FMXER_6R6URYBpV6TYPn9A,rV1nkTx58v_q7YGDkGzfxQ,5,5.0,2016-06-10,Las Vegas,where to start...\na few months ago I slammed....,Auto Repair;Automotive
7,7Je9L6bqTSBd4Ah9M3rvDw,C2VIYAElOUer5rBZH71c_A,5,4.5,2012-10-06,Phoenix,What a difference a new location with more spa...,Education;Auto Repair;Specialty Schools;Automo...
10,9QUixlmZlA_v1JSqBeZ2Uw,FV3v0F5yAsAZ6PVXFsVjeA,5,4.0,2016-11-09,Henderson,I am so glad I found this shop. I put off tak...,Body Shops;Auto Repair;Automotive


In [19]:
df[df['stars_x'] == 1].head(3)

,review_id,business_id,stars_x,stars_y,date,city,text,categories
0,JN0tmjiksWfr4i2NIFuPXw,pLZ9oZM8c6MNbRlg06lBPg,1,3.0,2016-08-03,Las Vegas,THEY SELL WRECKED VEHICLES! \nI purchased a 20...,Auto Repair;Car Dealers;Automotive;Auto Parts ...
2,R9p3ug8zNPvawYX_jsO0UA,Tknj3u-3CZTI-q52saTAIA,1,1.0,2012-09-12,Toronto,"If you want the ""I don't give a shit "" attitud...",Car Dealers;Auto Repair;Automotive
5,oLRl6PegGyjh7JA-8fUVzQ,btoBhgcY7vtvQ2M0itdTBQ,1,5.0,2016-06-10,Las Vegas,where to start...\na few months ago I slammed....,Auto Repair;Oil Change Stations;Automotive;Whe...


In [20]:
df['stars_x'].value_counts()
#Nombre de reviews très positives et très négatives

5    18166
1     8066
Name: stars_x, dtype: int64

In [21]:
ratio = float(len(df[df['stars_x'] == 5])) / (len(df[df['stars_x'] == 5]) + 
                                                                        len(df[df['stars_x'] == 1]))

In [22]:
print("Null Accuracy: %0.6f" % (ratio))

Null Accuracy: 0.692513


# Sentiment Analysis avec TextBlob

In [24]:
# print the first yelp review
print (df.text[0])

THEY SELL WRECKED VEHICLES! 
I purchased a 2010 Nissan Altima with 41,000 miles on it in 2013 from Planet Nissan. They charged me $18,000 and provided me with no warranty, except for manufacturer powertrain. Immediately after purchasing the vehicle I began having to make out of pocket repairs to the suspension, brakes and exhaust system every 6 to 9 months with absolutely no clue as to why after all the repairs the car still drove somewhat unsteady at high speeds. 
    Two years after I bought the car I was finally sick of the constant problems and decided to trade it in with another dealership, during the trade in process I found out that Planet Nissan had sold the vehicle to me knowing that it was involved in a "major" car accident which caused frame damage six months prior to them selling it to me.
Unfortunately there's not much I can do now, but I would advise any savvy consumer to avoid this dealership by all means possible and go someplace that is a bit more reputable!


In [27]:
df.stars_x[0]

1

In [30]:
# polarity goes from -1 (most negative) to 1 (most positive)
review = TextBlob(df.text[0])

In [42]:
review.sentiment.polarity

-0.006071428571428586

In [32]:
review.sentiment.subjectivity

0.6027705627705628

## Calculer le sentiment de chaque review avec Textblob

In [39]:
len(df)

26232

In [52]:
# create a new DataFrame column for sentiment
df['texblob_sentiment_score'] = df.text.apply(lambda x : TextBlob(x).sentiment.polarity)

In [55]:
# let's look at some negative reviews
df[df['stars_x'] == 1].head(10)

,review_id,business_id,stars_x,stars_y,date,city,text,categories,texblob_sentiment_score
0,JN0tmjiksWfr4i2NIFuPXw,pLZ9oZM8c6MNbRlg06lBPg,1,3.0,2016-08-03,Las Vegas,THEY SELL WRECKED VEHICLES! \nI purchased a 20...,Auto Repair;Car Dealers;Automotive;Auto Parts ...,-0.006071
2,R9p3ug8zNPvawYX_jsO0UA,Tknj3u-3CZTI-q52saTAIA,1,1.0,2012-09-12,Toronto,"If you want the ""I don't give a shit "" attitud...",Car Dealers;Auto Repair;Automotive,-0.237500
5,oLRl6PegGyjh7JA-8fUVzQ,btoBhgcY7vtvQ2M0itdTBQ,1,5.0,2016-06-10,Las Vegas,where to start...\na few months ago I slammed....,Auto Repair;Oil Change Stations;Automotive;Whe...,0.025052
6,cIeF6K2kjNNkJ2pMKrGztA,Gxu3_yzEAXKtkBB0YpnvlQ,1,1.5,2009-07-19,Peoria,Last weekend I decided to go on a hunt to see ...,Car Dealers;Automotive;Auto Repair,0.133164
13,1ZgiihHVMx1mFDDBn9HEwQ,JyANaAVhdRkl1ABMvbgNUw,1,1.0,2015-04-04,Las Vegas,I have gone here 3 times with my 3 different c...,Auto Repair;Automotive,0.200000
25,L6G91b3rVGPkcBITzAXB3w,_fACaVWH0WHC1yTu9vWJpQ,1,3.0,2013-06-15,Las Vegas,I had my car repaired there a year ago. I had ...,Auto Glass Services;Automotive;Body Shops;Auto...,-0.120000
37,4IkfGEhDvxPpmVdBWR7gwg,pDcU-HEnNr631YpwxAqYsQ,1,3.0,2012-10-12,Mesa,If only no stars were an option....\nNot only ...,Auto Repair;Automotive;Auto Parts & Supplies;C...,-0.121131
45,TCQuJYiQ-JNmcusBhV5t3Q,n5Bu0WBhCiQ5umCE5EzSjg,1,3.0,2017-10-28,Peoria,I came ready to buy a car but not a single per...,Car Dealers;Auto Repair;Used Car Dealers;Auto ...,-0.192143
47,-qmv0_kfTyj503zsZJUyww,h5tsd-0qKzaRB6w7LIx-3Q,1,1.5,2017-02-21,Phoenix,I was in the market for a new 2017 Nissan Sent...,Automotive;Auto Repair;Car Dealers,0.326144
59,2R0__embngOiugPH2DGIFA,h7l3xt_CAaFMD63ZgdXXWw,1,4.0,2015-06-04,Phoenix,Horrible customer service. More excuses than w...,Auto Repair;Automotive;Towing;Tires;Oil Change...,-0.056250


In [58]:
# let's check a random negative review
print (df.text[2])

If you want the "I don't give a shit " attitude from some of the service dept staff take you car for service here . Horrible experience from the service dept staff not all but all it takes is one. First a last time I take my car there. Avoid if you can


In [61]:
# let's look at some positive reviews
df[df['stars_x'] == 5].head(10)

,review_id,business_id,stars_x,stars_y,date,city,text,categories,texblob_sentiment_score
3,FMXER_6R6URYBpV6TYPn9A,rV1nkTx58v_q7YGDkGzfxQ,5,5.0,2016-06-10,Las Vegas,where to start...\na few months ago I slammed....,Auto Repair;Automotive,0.025052
7,7Je9L6bqTSBd4Ah9M3rvDw,C2VIYAElOUer5rBZH71c_A,5,4.5,2012-10-06,Phoenix,What a difference a new location with more spa...,Education;Auto Repair;Specialty Schools;Automo...,0.301388
10,9QUixlmZlA_v1JSqBeZ2Uw,FV3v0F5yAsAZ6PVXFsVjeA,5,4.0,2016-11-09,Henderson,I am so glad I found this shop. I put off tak...,Body Shops;Auto Repair;Automotive,0.226032
11,l8I-qUOx5ComFrhBE127hg,-1TUzO4XCmOeqi9eAGeDWA,5,4.5,2014-10-23,Champaign,"I was going through town on a road trip, and g...",Tires;Auto Repair;Automotive,0.131122
12,fhu9QLeHe50BHHEYmUOXjw,5Hm7p_B03gc_hIKvjriqLw,5,4.5,2014-08-07,Glendale,Very friendly atmosphere. First time I've been...,Auto Repair;Automotive,0.382500
14,GzAgB7-8poUnwrhiEHWEiQ,uRpjhhLclGlw_0oxDVicAw,5,4.0,2011-03-28,Madison,Although I am not planning on purchasing a BMW...,Automotive;Auto Parts & Supplies;Commercial Tr...,0.186002
16,E-M8zZdaqfrfnta3YtRpHw,WJ-HHlzkSTqlB6zwlP8x7A,5,3.5,2013-04-02,Middleton,I love Zimbrick Acura. I love every Zimbrick i...,Automotive;Auto Repair;Auto Parts & Supplies;C...,0.188243
17,LAPsOjIaeKsHPPC57YwFEQ,S_JZst0IGCads_KhoueMCw,5,4.5,2016-06-13,Las Vegas,Randy Clements is amazing at what he does !. M...,Mobile Dent Repair;Body Shops;Auto Detailing;A...,0.488241
18,6pLO85CJGwzpT2rGZ0D9NA,aK8DXte05wwI_9jmOrceXg,5,4.0,2013-11-21,Scottsdale,I have been to this location (Scottsdale Rd ju...,Automotive;Tires;Auto Repair,0.113062
20,5Wvh1T36wmTiURwIXJz7tw,SRZxlUuGact-r_AJ2fDEgA,5,4.0,2016-12-04,Charlotte,"Very quick, efficient service. I stopped in fo...",Automotive;Oil Change Stations;Auto Repair,0.102778


In [63]:
# let's check a random predicted positive review, 17 seems to tbe good candidate
print (df.text[17])

Randy Clements is amazing at what he does !. My 428i got rear ended and the bumper was cracked . Randy made it easy for me by dealing with the insurance company of who was at fault . He ordered my bumper and had my car ready in two days !. He did a Fantastic job. The car looks like new , it was washed when I picked it up. He is very pleasant to work with and cares about his customers . Randy I would give you 10 stars if I could . Metropolitan auto shop is a great place in general , very friendly staff and helpful . I will be coming here for all my car needs . Thank you !. :)


## Regarder la performance de Textblob

In [64]:
# reviews with most positive sentiment
df[df['texblob_sentiment_score'] == 1]['text'].head()

128     Excellent and helping people here! Will contin...
1187    The best car dealership in the state! But you ...
1647    Rocky Marshall at Prestige Service Dept. Was e...
1710    The Chapman BMW experience has been wonderful!...
2445    I always appreciate the honesty and the quickn...
Name: text, dtype: object

In [65]:
# reviews with most negative sentiment
df[df['texblob_sentiment_score'] == -1]['text'].head()

169     Absolutely horrible! If it wasn't for Vincent ...
1571    Worst car buying experience of my life - no on...
1689    Worst experience I Ever had buying a challener...
3002    Person complaining about a cable if that is th...
3823    avoid this guy if u care about your hard earne...
Name: text, dtype: object

In [67]:
# what is the minimum sentiment given by textblob for negative reviews?
df[df['stars_x'] == 1]['texblob_sentiment_score'].min()

-1.0

In [68]:
# what is the maximum sentiment given by textblob for negative reviews?
df[df['stars_x'] == 1]['texblob_sentiment_score'].max()

1.0

In [69]:
# positive sentiment in a 1-star review
df[(df['stars_x'] == 1) & (df['texblob_sentiment_score'] > 0.7)]

,review_id,business_id,stars_x,stars_y,date,city,text,categories,texblob_sentiment_score
8132,WapYI7aPMg7nrIEoQcNYhw,w3k4afCJEih9-kPL6ZI0Og,1,2.0,2017-03-13,Glendale,Update ....All is Well that Ends Well ..Sold M...,Auto Repair;Car Dealers;Automotive,0.976562
17257,gnZ269fA1FRli-2PrLFcKQ,WqzT6XTdSlt9IzNzyEcO4A,1,3.5,2015-04-07,Gilbert,NAPA Auto Care and owner Jack Griffith is noth...,Auto Repair;Automotive,1.000000
21182,MDSf1axnFMFAsTnsJamWbA,ali4IoPkincXzwHgWSk_rA,1,4.0,2016-06-07,Las Vegas,Great service before the sale but mass confusi...,Auto Repair;Used Car Dealers;Automotive;Car De...,0.800000
24352,TQVlNThXtIOIMNmCg_XFVA,ihCclCbKNG7yL6nkYTviEQ,1,4.0,2017-03-25,Tempe,Used to deal with Jason then he got fired. Ken...,Auto Parts & Supplies;Oil Change Stations;Auto...,0.900000
27769,0iuD7DAFlAlv3WecJa_RlQ,E48Fxo1H85mc2Vq0FcSnRQ,1,4.0,2016-08-16,Las Vegas,Don't worry Cesar I'll go ahead and upload the...,Automotive;Auto Repair;Body Shops,0.750000


In [72]:
print (df.text[8132])

Update ....All is Well that Ends Well ..Sold My Ride for $$$ (a lot more than offered) a bought a truck cash ..No Loan or Payments !!!


In [74]:
# what is the minimum sentiment given by textblob for positive reviews?
df[df['stars_x'] == 5]['texblob_sentiment_score'].min()

-1.0

In [75]:
# what is the maximum sentiment given by textblob for positive reviews?
df[df['stars_x'] == 5]['texblob_sentiment_score'].max()

1.0

In [76]:
# negative sentiment in a 5-star review
df[(df['stars_x'] == 5) & (df['texblob_sentiment_score'] < -0.3)]

,review_id,business_id,stars_x,stars_y,date,city,text,categories,texblob_sentiment_score
3002,tDrusWcZbE0V_QMxoQvEnw,u9-YcAtbCrsI-dx_mGvc_g,5,3.5,2015-10-12,Pittsburgh,Person complaining about a cable if that is th...,Automotive;Auto Repair,-1.000000
12866,1RPAyaDEbflbEZBykO8uWg,lyl3FDeeex8rV5WfREZdyQ,5,3.5,2016-06-16,Henderson,The Kia service department has always taken ca...,Car Dealers;Auto Repair;Automotive,-0.350000
15700,eVlcAMR19rTz5Nw5z6hulw,5HGO1CxM0qPIS_u4L90Bvw,5,5.0,2016-07-18,Las Vegas,Showed up without any appointment and less the...,Auto Repair;Automotive,-0.406901
18022,BXbe1L7VUDvuwZMBE129Ug,a9k9MQrr5KFyMD7_Lnwrhw,5,4.5,2016-10-03,Las Vegas,"In and out in a few minutes for a smog check, ...",Oil Change Stations;Smog Check Stations;Automo...,-0.312500
19733,d-NoG2yh64F7jR7cHcrZcQ,b9DbuRf1pyB9eDkF9Gdnhg,5,5.0,2016-12-16,Phoenix,David is a BMW expert. He knew what was wrong...,Automotive;Auto Repair,-0.355556
20031,VdVbSYowB0tsyeekp39RTQ,8vaXVv3PUNxGYDv1W5evEQ,5,2.5,2015-07-21,Phoenix,Darren helped me out big time today. I used th...,Oil Change Stations;Tires;Auto Repair;Automotive,-0.364583
23371,KMpsSae8Xl4Mhg_fET00aA,LwNZ1AR4_5iukgQYuhmTOg,5,3.5,2012-12-01,Las Vegas,"I NEVER do reviews, so I'll keep it short...\n...",Automotive;Auto Parts & Supplies;Auto Repair;C...,-0.468750
23847,oJSvq6ElZ71oQYZ05VdGmg,C7JaaErXLB3B3486LfWg5w,5,3.5,2014-03-14,Henderson,Usually in and out !! If I knew the girls name...,Car Dealers;Auto Parts & Supplies;Auto Repair;...,-0.953674


In [78]:
print (df['text'][3002])

Person complaining about a cable if that is the worst thing in your life then deal with it


## Use TextBlob to calculate stars for the reviews

In [79]:
df['textblob_predicted_stars'] = df['texblob_sentiment_score'].apply(lambda x : 5 if x > 0.1 else 1)

In [80]:
df.head(3)

,review_id,business_id,stars_x,stars_y,date,city,text,categories,texblob_sentiment_score,textblob_predicted_stars
0,JN0tmjiksWfr4i2NIFuPXw,pLZ9oZM8c6MNbRlg06lBPg,1,3.0,2016-08-03,Las Vegas,THEY SELL WRECKED VEHICLES! \nI purchased a 20...,Auto Repair;Car Dealers;Automotive;Auto Parts ...,-0.006071,1
2,R9p3ug8zNPvawYX_jsO0UA,Tknj3u-3CZTI-q52saTAIA,1,1.0,2012-09-12,Toronto,"If you want the ""I don't give a shit "" attitud...",Car Dealers;Auto Repair;Automotive,-0.237500,1
3,FMXER_6R6URYBpV6TYPn9A,rV1nkTx58v_q7YGDkGzfxQ,5,5.0,2016-06-10,Las Vegas,where to start...\na few months ago I slammed....,Auto Repair;Automotive,0.025052,1


In [82]:
print ('Textblob Accuracy', metrics.accuracy_score(df['stars_x'], df['textblob_predicted_stars']))

Textblob Accuracy 0.8584172003659652
